In [1]:
# Импорт библиотек
import os
import time
import cotel
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root

# Основные константы
calctolerance = 10**-3
calcmethod = "hybr"
KPDnasos = 0.8074
KPD = 0.99

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streamsZAD = pd.read_excel(
    "streams.xlsx", sheet_name="water", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)


# Состав газов в номинале
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = (
    gas_streams0.at["GTU-PEVD", "N2"],
    gas_streams0.at["GTU-PEVD", "O2"],
    gas_streams0.at["GTU-PEVD", "CO2"],
    gas_streams0.at["GTU-PEVD", "H2O"],
    gas_streams0.at["GTU-PEVD", "Ar"],
)

gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure= gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies


####################################################################################################
# Расчет ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, GTU_input, "GTU-KU")
Gas_turbine_res = Gas_turbine.calc()
electric.loc["GTU", "N":"KPD"] = [Gas_turbine_res["N"], Gas_turbine_res["eff"]]
electric.loc["DK", "N"] = Gas_turbine_res["Ndk"]
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
gas_streams.loc["GTU-KU", "T"] = Gas_turbine_res["T"]
gas_streams.loc["GTU-KU", "G"] = Gas_turbine_res["G"]
gas_streams.loc["GTU-KU", "P"] = 0.1
Gas_turbine_composition=pd.read_excel("input.xlsx", sheet_name="Gas_composition0", index_col=0)

# чтение с вводного экселя входных данных
Pvd = water_streamsZAD.loc["IVD-PEVD", "P"]
Pnd = water_streamsZAD.loc["IND-PPND", "P"]
Twat0 = water_streams0.loc["REC-GPK", "T"]

#Параметры газа на выходе в КУ
gas_streams.loc["GTU-PEVD", "T":"P"] = gas_streams.loc["GTU-KU", "T":"P"]
gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
gas_streams.loc["GTU-PEVD", "N2":"Ar"]=Gas_turbine_composition.loc["Fraction", "N2":"Ar"]

# Состав газов при частичной нагрузке
fractiongas = (
    gas_streams.at["GTU-PEVD", "N2"],
    gas_streams.at["GTU-PEVD", "O2"],
    gas_streams.at["GTU-PEVD", "CO2"],
    gas_streams.at["GTU-PEVD", "H2O"],
    gas_streams.at["GTU-PEVD", "Ar"],
)
gas1 = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)

# Задание энтальпий газа в рабочем режиме
Temperatures = gas_streams.loc["GTU-KU":"GPK-out", "T"]
Pressure= gas_streams.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas1.p_t(Pressure, x)["h"], Temperatures))
gas_streams.loc["GTU-KU":"GPK-out", "H"] = Enthalpies

# Задание в раб табл параметров по воде
water_streams.loc["IVD-PEVD":"PEN-EVD", "P"] = Pvd
water_streams.loc["IND-PPND":"REC-GPK", "P"] = Pnd
water_streams.loc["BND-PEN", "P"] = Pnd
water_streams.loc["REC-GPK", "T"] = Twat0

# Расчёт параметров на насыщении в испарительных поверхностях (в 1 итерации приближение: недогрев до н.п. на входе 3°С)
water_streams.loc["IVD-PEVD", "T"] = water.p_q(Pvd, 1)["T"]
water_streams.loc["IVD-PEVD", "H"] = water.p_q(Pvd, 1)["h"]
water_streams.loc["IND-PPND", "T"] = water.p_q(Pnd, 1)["T"]
water_streams.loc["IND-PPND", "H"] = water.p_q(Pnd, 1)["h"]
water_streams.loc["EVD-IVD", "T"] = water.p_q(Pvd, 1)["T"] - 1
water_streams.loc["GPK-IND", "T"] = water.p_q(Pnd, 1)["T"] - 1
water_streams.loc["EVD-IVD", "H"] = water.p_t(Pvd, water_streams.loc["EVD-IVD", "T"])[
    "h"
]
water_streams.loc["GPK-IND", "H"] = water.p_t(Pnd, water_streams.loc["GPK-IND", "T"])[
    "h"
]

####################################################################################################
# расчет котла целиком
Whole_cotel = cotel.cotel_all(
    KPD,
    KPDnasos,
    calctolerance*10**2,
    gas0,
    gas1,
    water,
    calcmethod,
    gas_streams0,
    water_streams0,
    gas_streams,
    water_streams,
)
Whole_cotel.calc(maxiterations=50)


Whole_cotel = cotel.cotel_all(
    KPD,
    KPDnasos,
    calctolerance,
    gas0,
    gas1,
    water,
    calcmethod,
    gas_streams0,
    water_streams0,
    gas_streams,
    water_streams,
)
Whole_cotel.calc(maxiterations=50)


print(
    "Степень сухости пара в ЭВД: ",
    water.p_h(water_streams.at["EVD-IVD", "P"],
              water_streams.at["EVD-IVD", "H"])["Q"],
)
print(
    "Степень сухости пара в ГПК: ",
    water.p_h(water_streams.at["GPK-IND", "P"],
              water_streams.at["GPK-IND", "H"])["Q"],
)

NameError: name 'prop1' is not defined

In [ ]:
# Вывод газовых потоков
gas_streams

In [ ]:
# Вывод водяных потоков
water_streams['PEVD-DROSVD':'REC-GPK']

In [ ]:
water_streams0['PEVD-DROSVD':'REC-GPK']

In [ ]:
Errors_water_T= abs(water_streams['PEVD-DROSVD':'REC-GPK']['T'] -water_streams0['PEVD-DROSVD':'REC-GPK']['T'])/water_streams0['PEVD-DROSVD':'REC-GPK']['T']*100
Errors_water_G= abs(water_streams['PEVD-DROSVD':'REC-GPK']['G'] -water_streams0['PEVD-DROSVD':'REC-GPK']['G'])/water_streams0['PEVD-DROSVD':'REC-GPK']['G']*100

Errors_water_G

In [ ]:
# Невязка по всем параметрам
Qgas = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["PEVD-IVD", "H"])
)
Qwat = water_streams.at["PEVD-DROSVD", "G"] * (
    water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["IVD-PEVD", "H"]
)
print("dQ/Q ПЕВД+ИВД", ((Qgas - Qwat) / Qgas * 100))

Qgas1 = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["IVD-EVD", "H"])
)
Qwat1 = water_streams.at["PEVD-DROSVD", "G"] * (
    water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["EVD-IVD", "H"]
)
print("dQ/Q ПЕВД+ИВД+ЭВД", ((Qgas1 - Qwat1) / Qgas1 * 100))

Qgas = (
    KPD
    * gas_streams.at["EVD-PPND", "G"]
    * (gas_streams.at["EVD-PPND", "H"] - gas_streams.at["IND-GPK", "H"])
)
Qwat = (
    water_streams.at["IND-PPND", "G"]
    * (water_streams.at["IND-PPND", "H"] - water_streams.at["GPK-IND", "H"])
    + water_streams.at["PPND-DROSND", "G"]
    * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["IND-PPND", "H"])
    + water_streams.at["BND-PEN", "G"]
    * (water_streams.at["BND-PEN", "H"] - water_streams.at["GPK-IND", "H"])
)
print("dQ/Q ППНД+ИНД", ((Qgas - Qwat) / Qgas * 100))

Qgas1 = (
    KPD
    * gas_streams.at["EVD-PPND", "G"]
    * (gas_streams.at["EVD-PPND", "H"] - gas_streams.at["GPK-out", "H"])
)
Qwat1 = (
    water_streams.at["GPK-IND", "G"]
    * (water_streams.at["GPK-IND", "H"] - water_streams.at["X-GPK", "H"])
    + water_streams.at["IND-PPND", "G"]
    * (water_streams.at["IND-PPND", "H"] - water_streams.at["GPK-IND", "H"])
    + water_streams.at["PPND-DROSND", "G"]
    * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["IND-PPND", "H"])
    + water_streams.at["BND-PEN", "G"]
    * (water_streams.at["BND-PEN", "H"] - water_streams.at["GPK-IND", "H"])
)
print("dQ/Q ППНД+ИНД+ГПК", (Qgas1 - Qwat1) / Qgas1 * 100)

Qgasall = (
    KPD
    * gas_streams.at["GTU-PEVD", "G"]
    * (gas_streams.at["GTU-PEVD", "H"] - gas_streams.at["GPK-out", "H"])
)
Qwatall = (
    water_streams.at["PPND-DROSND", "G"]
    * (water_streams.at["PPND-DROSND", "H"] - water_streams.at["X-GPK", "H"])
    + water_streams.at["PEVD-DROSVD", "G"]
    * (water_streams.at["PEVD-DROSVD", "H"] - water_streams.at["X-GPK", "H"])
    - water_streams.at["BND-PEN", "G"]
    * (water_streams.at["PEN-EVD", "H"] - water_streams.at["BND-PEN", "H"])
)
print("dQ/Qsumm", (Qgasall - Qwatall) / Qgasall * 100)

In [ ]:
gas_streams0

In [ ]:
water_streams0

In [ ]:
electric

In [ ]:
heaters

In [ ]:
GTU_input